In [14]:
import numpy as np
from typing import List
from random import randint
import math

In [15]:
def ReadFile(Fname):
    with open(Fname) as f:
    
        row = [int(x) for x in next(f).split()] # read first line
        empty = [int(x) for x in next(f).split()] # read next line
        col = [int(x) for x in next(f).split()] # read next line
        empty = [int(x) for x in next(f).split()] # read next line
        empty = [int(x) for x in next(f).split()] # read next line

        Utility = []
        Utility.append([float(x) for x in next(f).split()])

        Utility =Utility[0]

        empty = [int(x) for x in next(f).split()] # read next line

        A = []
        count = 0
        while count < row[0]: # read rest of lines
            A.append([float(x) for x in next(f).split()])
            count = count + 1
    
        empty = [int(x) for x in next(f).split()] # read next line

        RHS = []
        RHS.append([float(x) for x in next(f).split()])
        RHS =RHS[0]

    RHS = np.array(RHS)
    A = np.array(A)
    Utility = np.array(Utility)
    row, col = A.shape
    
    return np.array(A),np.array(RHS),row,col,np.array(Utility)

In [16]:
#Define a função objetivo
def fo(c,x):
    return sum(c*x)


#Verifica a viabilidade da solução
def viabilidade(a,b,x):
    factivel= True
    
    for i in range(len(a)):
        if sum(a[i]*x) > b[i]:
            factivel= False 
            break
    return factivel

#print (fo,viabilidade)

In [17]:
#função para calcular penalidade
def penalidade(x, A, RHS):
    i=0
    m=0
    while i < len(A):
        m += max(sum(A[i] * x) - RHS[i], 0 )
        i=i+1
    return m

In [18]:
#função fitness- aptidão
def fitness(fo, penalidade):
    fit= fo-penalidade
    return fit 

In [19]:
#seleção dos pais para gerar os filhos sorte de uma quatidade aleatoria
def selecaoReproducao(populacao,PercentualTorneio,A,RHS,geracao):
    sorteados = np.zeros(int(len(populacao)*PercentualTorneio))
    sorteados[:] = np.nan
    i= 0
    while i < len(sorteados):
        posicao_sorteada= randint(0,len(populacao)-1)
        if posicao_sorteada in sorteados :
            i = i
        else:
            sorteados[i] = posicao_sorteada
            i= i + 1

    pai_posicao = -1
    pai_aptidao = -1                 
    mae_posicao = -1
    mae_aptidao = -1

    i=0                  
    while i<len(sorteados):
        x = populacao[int(sorteados[i])]
        _penalidade = penalidade(x, A, RHS)
        _fo = fo(Utility,populacao[int(sorteados[i])])
        _fitness = fitness(_fo,_penalidade)

        if _fitness > pai_aptidao :
            pai_aptidao = _fitness
            pai_posicao = sorteados[i]
        i= i + 1

    sorteados = np.zeros(int(len(populacao)*PercentualTorneio))
    sorteados[:] = np.nan    
    i =0
    while i < len(sorteados):
        posicao_sorteada = randint(0,len(populacao)-1)
        if posicao_sorteada == pai_posicao or posicao_sorteada in sorteados :
            i = i
        else:
            sorteados[i] = posicao_sorteada
            i= i + 1

    i=0                  
    while i<len(sorteados):
        _penalidade = penalidade(populacao[int(sorteados[i])], A, RHS)
        _fo = fo(Utility,populacao[int(sorteados[i])])
        _fitness = fitness(_fo,_penalidade)                      
        if _fitness > mae_aptidao :
            mae_aptidao = _fitness
            mae_posicao = sorteados[i]
        i= i + 1

    #print("Pai = F:",pai_aptidao,"P:",pai_posicao)
    #print("Mae = F:",mae_aptidao,"P:",mae_posicao)

    #==========================================================
    #crossover Reproducao
    filho_um = np.array([])
    filho_dois = np.array([])

    porcentagem = randint(0,100)

    for i  in range(len(populacao[int(pai_posicao)])):
        aleatorio =randint(0,100)
        if aleatorio >= 60:
            filho_um = np.insert(filho_um,len(filho_um),(1 - populacao[int(pai_posicao)][i])) 
            filho_dois = np.insert(filho_dois,len(filho_dois),(populacao[int(mae_posicao)][i]))
        else:    
            filho_um = np.insert(filho_um,len(filho_um),(populacao[int(pai_posicao)][i]))
            filho_dois = np.insert(filho_dois,len(filho_dois),(1 - populacao[int(mae_posicao)][i]))
    
    if randint(0,100) <=3:
        posicao_mutacao = randint(0,len(filho_um)-1)
        filho_mutado = randint(1,2)
        if filho_mutado == 1:
            filho_um[int(posicao_mutacao)]=1
        else:
            filho_dois[int(posicao_mutacao)]=1
        #print(geracao,"mutou")
    return filho_um,filho_dois  
        


In [20]:
def NovaGeracao(populacao,A,RHS,geracao):
    NovaPopulacao= np.array([])
    qtd_torneios = int(len(populacao)/2)

    for i  in range(qtd_torneios):
        PercentualTorneio= randint(40,80)/100
        filho_um,filho_dois = selecaoReproducao(populacao,PercentualTorneio,A,RHS,geracao)
        if len(NovaPopulacao)==0:
            NovaPopulacao = filho_um
            NovaPopulacao = np.vstack((NovaPopulacao,filho_dois))
        else:
            NovaPopulacao = np.vstack((NovaPopulacao,filho_um))
            NovaPopulacao = np.vstack((NovaPopulacao,filho_dois))
        #print("NovaGeracao:",i, "Torneios:", qtd_torneios)

    return NovaPopulacao
    

In [21]:
def pop_inicial(Tamanho,cols,A,RHS):
    #print("tamano pop inicial",Tamanho)
    populacao=np.array([])
    for i in range(Tamanho):
        while True:
            individuo = np.zeros(cols)
            for j in range(cols):
                posicao = randint(0,cols-1)
                individuo[int(posicao)] = 1 - individuo[int(posicao)]
            
            if viabilidade(A,RHS,individuo):
                if len(populacao)==0:
                    populacao = individuo
                else:
                    populacao = np.vstack((populacao,individuo))
                break
            
    return populacao


In [22]:
def GeneticAlgoritm(A,RHS,rows,cols,Utility,qtddegeracoes,tamnhoDapopulacao):
    populacao = pop_inicial(int(cols*0.6),cols,A,RHS)
    #print("populacao inicial",populacao)
    FO = 0
    x = np.array([])
    for i  in range(tamnhoDapopulacao):
        populacao = NovaGeracao(populacao,A,RHS,i)
        print("geracao",i)
        for xx in populacao:
            _fo = fo(Utility,xx)
            if _fo>FO:
                if viabilidade(A,RHS,xx):
                    FO=_fo
                    x = xx.copy()
                    print("FO:",FO)

        #print(populacao)

    print("*FO:",FO)
    print("*X:",x)

    
    
          

In [23]:
C5I10 = ""
C5I10 = r"Dados\Knap_C5I10.dat"
A,RHS,row,col,Utility = ReadFile(C5I10)
GeneticAlgoritm(A,RHS,row,col,Utility,100,10)
C30I20 = ""
C30I20 = r"Dados\Knap_C30I20.dat"
A,RHS,row,col,Utility = ReadFile(C30I20)
GeneticAlgoritm(A,RHS,row,col,Utility,100,10)
C150I200 = ""
C150I200 = r"Dados\Knap_C150I200.dat"
A,RHS,row,col,Utility = ReadFile(C150I200)
GeneticAlgoritm(A,RHS,row,col,Utility,150,10)

geracao 0
FO: 2.61447
geracao 1
FO: 3.14066
geracao 2
geracao 3
FO: 3.46551
geracao 4
FO: 3.51766
geracao 5
geracao 6
geracao 7
geracao 8
geracao 9
*FO: 3.51766
*X: [1. 1. 1. 0. 1. 0. 0. 1. 1. 1.]
geracao 0
FO: 4.1949
FO: 6.82526
geracao 1
geracao 2
geracao 3
FO: 6.84149
geracao 4
geracao 5
geracao 6
FO: 7.11066
geracao 7
FO: 7.85505
geracao 8
geracao 9
*FO: 7.85505
*X: [0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0.]
geracao 0
FO: 47.951580000000014
FO: 56.76264000000001
FO: 57.46847999999999
FO: 57.81537999999997
FO: 58.278309999999976
FO: 61.56227000000001
FO: 64.40780000000001
geracao 1
geracao 2
geracao 3
FO: 64.63123999999999
geracao 4
geracao 5
geracao 6
FO: 66.14568000000001
geracao 7
geracao 8
geracao 9
*FO: 66.14568000000001
*X: [1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0.
 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0.
 1. 1. 0. 0. 1. 1. 